<a href="https://colab.research.google.com/github/shanmcm/Sentiment-Analysis-Amazon-Musical-Instruments-Reviews/blob/main/Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pickle5
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import os
import random
import pandas as pd
from torchtext.legacy import data
import pickle5 as pickle
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import params
import net
import numpy as np
from dataset import AmazonDataset
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
import dataset
import net
import params
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def train_val_dataset(d, val_split=0.25):
    train_idx, val_idx = train_test_split(list(d.data.index), test_size=val_split)
    return Subset(d, train_idx), Subset(d, val_idx)

In [5]:
dataset = AmazonDataset()
dataset.load_dataset()
dataset.my_filter()
train_dataset, test_dataset = train_val_dataset(dataset)

In [6]:
type(train_dataset[0][0])

list

In [7]:
torch.randn(3,2)

tensor([[ 0.0124, -0.6572],
        [ 0.5605, -0.5754],
        [ 0.8085, -0.2907]])

In [8]:
torch.randn(4,2)

tensor([[-0.4053,  0.9873],
        [-0.0549,  0.1335],
        [ 0.6608, -1.5051],
        [-0.8374, -0.0481]])

In [9]:
torch.randn(5,2)

tensor([[-0.3954,  0.3709],
        [ 0.3078, -0.6092],
        [ 0.6654, -1.0872],
        [ 1.8260, -1.3132],
        [ 1.3139,  0.4920]])

In [10]:
tmp = train_dataset


In [11]:
l = [torch.randn(3,2), torch.randn(4,2), torch.randn(5,2)] 
# nel nostro caso l = [recensione1, recensione2, ...],
# quindi nel caso esempio la lunghezza di ogni parola nella recensione è 2, nel nostro in questo case specifico è come se ogni lunghe
print(l)
print("----")
out = torch.nn.utils.rnn.pad_sequence(l, batch_first=True)
print(out)
print("----")
l_final = torch.split(out, [1,1,1])
print(l_final)
print("----")
'''
# out will now be a tensor with shape (3, 5, 2)
# You can transpose it back to (3, 2, 5) with
out = out.transpose(1, 2)
print(out)
'''

[tensor([[ 1.3914, -0.5865],
        [-1.2992, -1.0501],
        [-0.7417, -1.2190]]), tensor([[-0.2214,  1.5253],
        [-0.4063, -0.4586],
        [ 0.5411,  0.3608],
        [-0.1357, -0.1388]]), tensor([[-1.0698,  0.2527],
        [-1.0860, -0.2793],
        [-0.3274, -1.2031],
        [ 0.5366,  0.0464],
        [-0.2293,  0.6556]])]
----
tensor([[[ 1.3914, -0.5865],
         [-1.2992, -1.0501],
         [-0.7417, -1.2190],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.2214,  1.5253],
         [-0.4063, -0.4586],
         [ 0.5411,  0.3608],
         [-0.1357, -0.1388],
         [ 0.0000,  0.0000]],

        [[-1.0698,  0.2527],
         [-1.0860, -0.2793],
         [-0.3274, -1.2031],
         [ 0.5366,  0.0464],
         [-0.2293,  0.6556]]])
----
(tensor([[[ 1.3914, -0.5865],
         [-1.2992, -1.0501],
         [-0.7417, -1.2190],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]]), tensor([[[-0.2214,  1.5253],
         [-0.4063, -0.4586],

'\n# out will now be a tensor with shape (3, 5, 2)\n# You can transpose it back to (3, 2, 5) with\nout = out.transpose(1, 2)\nprint(out)\n'

In [12]:
from sklearn.metrics import accuracy_score
dataset.maximum_embedding_len = 500

# Define parameters
batch_size = params.BATCH_SIZE
hidden_dim = 128
embedding_size = len(dataset.__getitem__(0)[0])  # lunghezza embedding (selezionato uno a caso perchè tanto tutti hanno la stessa dimensione)
dropout_rate = params.DROPOUT_RATE
lr = params.LR
epochs = params.NUM_EPOCHS
best_loss = float('inf')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)  # , collate_fn=None, pin_memory=False)
# Build the model
lstm_model = net.SentimentAnalysis(batch_size,
                                   hidden_dim,
                                   embedding_size,
                                   dropout_rate)  # modificato LSTM con SentimentAnalysis (nome rete)

# optimization algorithm
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

to_train = True

ce = nn.CrossEntropyLoss()
mse = nn.MSELoss()
# train and validate
if to_train:
    for epoch in range(epochs):
        # training
        epoch_loss = 0
        epoch_acc = 0
        for idxs, (batch, labels) in enumerate(train_loader):
            #print(batch)
            batch = torch.stack(batch) #torch.from_numpy(batch)
            if idxs == 0:
              print(batch)
            #padded_data = pad_sequence(batch, batch_first=True, padding_value=0)
            optimizer.zero_grad()
            predictions = lstm_model(batch)  # (text, text_lengths) # batch_size, hidden_dim, vocab_size, window, dropout_rate
            loss = 0.5 * ce(predictions, labels) + 0.5 * mse(predictions, labels)
            #winners = predictions.argmax(dim=1)
            #corrects = np.where(winners == batch.labels)
            #accuracy = corrects.sum().float() / float(batch.labels.size(0))

            accuracy = accuracy_score(predicitons, labels)
            # perform backpropagation
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += accuracy.item()

        train_loss, train_acc = epoch_loss / len(train_loader), epoch_acc / len(train_loader)

        # save best model
        if train_loss < best_loss:
            best_valid_loss = train_loss
            torch.save(lstm_model.state_dict(), 'saved_weights_BiLSTM.pt')

        print(f'\tEpoch; {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')

# load weights and make predictions
lstm_model.load_state_dict(torch.load("saved_weights_BiLSTM.pt"))
epoch_loss = 0
epoch_acc = 0

lstm_model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)  # , collate_fn=None, pin_memory=False)

with torch.no_grad():
    for idx, batch in test_loader:
        text, text_lengths = batch.text

        predictions = lstm_model(batch).squeeze(1)

        loss = 0.5 * ce(predictions, batch.labels) + 0.5 * mse(predictions, batch.labels)

        winners = predictions.argmax(dim=1)
        corrects = (winners == batch.labels)
        accuracy = corrects.sum().float() / float(batch.labels.size(0))

        epoch_loss += loss.item()
        epoch_acc += accuracy.item()

test_loss, test_acc = epoch_loss / len(test_iterator), epoch_acc / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

tensor([[[ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0462,  0.0173,  0.0558,  ..., -0.0424,  0.0095,  0.0594],
         ...,
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000]],

        [[-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         ...,
         [ 0.0957, -0.2721,  1.3933,  ..., -0.6428, -0.1246, -0.6490],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000]],

        [[ 0.0091, -0.0929,  0.1970,  ..., -0.1409,  0.7560, -0.6236],
         [ 0.0171, -0.0574,  0.0526,  ..., -0

RuntimeError: ignored

Problemi:
1. Stiamo passando alla rete le frasi di testo (dataset.data) non tensori. In che formato vanno passati al modello? getitem restituisce una lista di elementi, dove ogni elemento è un tensore (ogni tensore corrisponde ad una parola della recenzione e ha lunghezza 3072) e la lunghezza della recensione è variabile
2. Attention weight va ancora chiamato
3. In dataset.embedded_wordataset_dict ci sono parole (keys) con "##" davanti, credo non ci debbano essere? (vedi cella sotto)

In [ ]:
embedded_data = dataset.format_data()

In [ ]:
lengths = [lambda x : len(x) for x in dataset.data.split("")]
print(lengths)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset.embedded_data, dataset.labels, train_size=0.7, random_state=0, shuffle = False)


training_set = DatasetMapper(x_train, y_train)
test_set = DatasetMapper(x_test, y_test)

batch_size = params.BATCH_SIZE

loader_training = DataLoader(training_set, batch_size=batch_size)
loader_test = DataLoader(test_set)

In [ ]:
'''
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.labels, train_size=0.7, random_state=0, shuffle = True)

train_data = { 'features': x_train, 'labels': y_train}
test_data = {'features': x_test, 'labels': y_test}

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
'''

In [ ]:
# Define parameters
batch_size = params.BATCH_SIZE
hidden_dim = 128
embedding_size = len(dataset.__getitem__(0)[0]) #lunghezza embedding (selezionato uno a caso perchè tanto tutti hanno la stessa dimensione)
dropout_rate = params.DROPOUT_RATE
lr = params.LR
epochs = params.NUM_EPOCHS
best_loss = float('inf')

# BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
# x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, train_size=0.8, random_state=0)
'''
train_iterator, test_iterator = data.BucketIterator.splits((train_data, test_data),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text), # Sort the batches by text length size
    sort_within_batch = True,
    device = device)

train_iterator = DataLoader(train_data, batch_size=batch_size)
test_iterator = DataLoader(test_data, batch_size = batch_size)
'''
# Build the model
lstm_model = net.SentimentAnalysis(batch_size,
                                   hidden_dim,
                                   embedding_size,
                                   dropout_rate) #modificato LSTM con SentimentAnalysis (nome rete)

# optimization algorithm
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

In [ ]:
loader_training.dataset[9]

In [ ]:
to_train = True

# train and validate
if (to_train):

    for epoch in range(epochs):

        # training 
        epoch_loss = 0
        epoch_acc = 0
        for batch in loader_training:
            optimizer.zero_grad()
            # retrieve text and no. of wordataset
            #text, text_lengths = batch.text
            
            predictions = lstm_model(batch)    #(text, text_lengths) # batch_size, hidden_dim, vocab_size, window, dropout_rate
            loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels.squeeze())+ 0.5*nn.MSELoss(predictions, batch.labels.squeeze())
            '''
            winners = predictions.argmax(dim=1)
            corrects = (winners == batch.labels)
            accuracy = corrects.sum().float() / float(batch.labels.size(0))
            '''
            accuracy=accuracy_score(kept_labels, predictions)

            # perform backpropagation
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
            
        train_loss, train_acc = epoch_loss / len(train_iterator), epoch_acc / len(train_iterator)

        # save best model
        if train_loss < best_loss:
            best_valid_loss = train_loss
            torch.save(lstm_model.state_dict(), 'saved_weights_BiLSTM.pt')

        print(f'\tEpoch; {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')  

In [ ]:
# load weights and make predictions
lstm_model.load_state_dict(torch.load("saved_weights_BiLSTM.pt"))
epoch_loss = 0
epoch_acc = 0

lstm_model.eval()

with torch.no_grad():
    for batch in test_iterator:
        text, text_lengths = batch.text

        predictions = lstm_model(batch).squeeze(1)

        loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels)+ 0.5*nn.MSELoss(predictions, batch.labels)

        winners = predictions.argmax(dim=1)
        corrects = (winners == batch.labels)
        accuracy = corrects.sum().float() / float(batch.labels.size(0))

        epoch_loss += loss.item()
        epoch_acc += accuracy.item()

test_loss, test_acc =  epoch_loss / len(test_iterator), epoch_acc / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')